#asdasd

In [ ]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [ ]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [ ]:
print(train_df.shape)

(598, 2881)


In [ ]:
train_x.head()

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.34,40.89,32.56,34.09,77.77,0.0,0.0,0.0,0.0,0.0
1,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.89,42.82,43.92,35.34,72.55,0.0,0.0,0.0,0.0,0.0
2,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.19,36.65,42.47,36.53,78.35,0.0,0.0,0.0,0.0,0.0
3,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,37.74,39.17,52.17,30.58,71.78,0.0,0.0,0.0,0.0,0.0
4,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.70,41.89,46.93,33.09,76.97,0.0,0.0,0.0,0.0,0.0


In [ ]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class','Y_Quality'])
train_y = train_df['Y_Quality']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [ ]:
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [ ]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


In [ ]:
mm = MinMaxScaler()
fitted = mm.fit(train_y.values.reshape(-1,1))
output = mm.transform(train_y.values.reshape(-1,1))
train_y = output.reshape(-1)
min_df, max_df = mm.transform([[0.525086],[0.534843]]).reshape(-1)

In [ ]:
# RF = RandomForestClassifier(random_state=37).fit(train_x, train_y)
RF = Lasso(alpha = 0.00001,random_state = 37, normalize=True).fit(train_x,train_y)
print('Done.')

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


Done.


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.584e-01, tolerance: 5.377e-04
  model = cd_fast.enet_coordinate_descent(


In [ ]:
preds = RF.predict(test_x)
print('Done.')

Done.


In [ ]:
pred_np = []
for i in range(len(preds)):
  pred_class = 0
  train_class = 0
  if preds[i] < min_df:
    pred_class = 0
  elif preds[i] > max_df:
    pred_class = 2
  else:
    pred_class = 1
  pred_np.append(pred_class)


In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['Y_Class'] = pred_np
submit.to_csv('./baseline_submission.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (10,6))
plt.scatter(range(len(test_y)),test_y)
plt.scatter(range(len(preds)),preds)
plt.plot(range(len(preds)),[min_df for i in range(len(preds))])
plt.plot(range(len(preds)),[max_df for i in range(len(preds))])
plt.show()

NameError: ignored

<Figure size 720x432 with 0 Axes>